### 获取订单号的参数

```python

qm = QHJMall(MALL_KEY)
qm.get_order(order_type,**kargs)


# 请求类型 0 表示所有订单 1 未发货  2 已发货 3 已完成 4 申请退款 5 退款成功  10 未支付
"order_status": kargs.get("order_status", 0),
"order_time": kargs.get("order_time", ""), # today yesterday week month
"order_id": kargs.get("order_id", ""), # 订单号
"order_type": order_type,  # original free
"page": kargs.get("page", 1),  #

```


In [1]:
import re
import sys
sys.path.append(r"D:\往期\QHJ\ZERO")
sys.path.append(r"E:\dataparse\Python_DATA_PARSE\QHJ\ZERO")
import warnings
warnings.filterwarnings("ignore")

from pyquery import PyQuery as pq
import requests
from urllib.parse import urlencode,parse_qs
import numpy as np
import pandas as pd
import redis

import json
from tools import *

qm = QHJMall(MALL_KEY)


import datetime



In [2]:
# 正式
conn = redis.Redis(**REDIS_MALL_ORDER_DIC)

# 测试



In [3]:
sku_goods = pd.read_excel(r'D:\奇货居\素材\商城图片素材\商品信息.xlsx',sheet_name='商品详情') \
                [['商品ID','发货商ID','goods_type','规格','单位', '市场价','售价','商品名简称']] \
                .rename(columns={"售价":"单价","商品名简称":"商品名"}).fillna({"goods_type":"original"})

free_goods = sku_goods[sku_goods['goods_type']=='free']

order_kargs = {
    "order_status":1,
    "order_time":"yesterday"
}

yd_data = qm.ext_order(order_kargs,sku_goods,free_goods)

order_kargs = {
    "order_status":1,
    "order_time":"today"
}
to_data = qm.ext_order(order_kargs,sku_goods,free_goods)

请求 {'order_status': 1, 'order_time': 'yesterday', 'order_id': '', 'order_type': 'original', 'page': 1}
请求 {'order_status': 1, 'order_time': 'yesterday', 'order_id': '', 'order_type': 'free', 'page': 1}
请求 {'order_status': 1, 'order_time': 'yesterday', 'order_id': '', 'order_type': 'free', 'page': 3}
请求 {'order_status': 1, 'order_time': 'yesterday', 'order_id': '', 'order_type': 'free', 'page': 2}
请求 {'order_status': 1, 'order_time': 'today', 'order_id': '', 'order_type': 'original', 'page': 1}
请求 {'order_status': 1, 'order_time': 'today', 'order_id': '', 'order_type': 'free', 'page': 1}


In [4]:

def uprow(row,):
    # 判断是否在已经 发过此物
    if  conn.sismember('order:old' ,row['订单号']):
        row['订单号'] = None
        
    return row



save_dir = r"D:\Downloads\QHJ_MALL"
save = SaveXl(save_dir)

data = pd.concat([to_data.apply(uprow,axis=1), #
                  yd_data.apply(uprow,axis=1)])
data.dropna(subset=['订单号'],inplace=True)
data.drop_duplicates(subset='订单号',inplace=True)

In [5]:
len(data)

12

In [6]:



save(data,"订单")
%run D:\往期\QHJ\ZERO\app\process_order.py


已存入 D:\Downloads\QHJ_MALL\订单 2019-12-16 15_46_18.xlsx
读取供应商信息
读取 D:\Downloads\QHJ_MALL\订单 2019-12-16 15_46_18.xlsx
>>>>系统字段统一为本地字段
>>>>连接供应商信息
存储订外发单信息
>>>>备份

存储新订单 D:\奇货居\work\外发订单\新订单
['订单号', '商品ID', '供应商', '商品名', '数量', '规格', '单位', '收件人', '收货地址', '联系方式', '备注', '发货商']
>>>>正在存储 三金_新订单 2019-12-16 15_46_18.xlsx
新订单存储完成

>>>>打开宏表
>>>>开始运行
>>>>关闭宏表
>>>>关闭完成

4.005s



In [7]:


stop


NameError: name 'stop' is not defined

In [ ]:
spp_path =  os.path.join(r"D:\奇货居\素材\商城图片素材","发货商详情.xlsx")
spp_data = pd.read_excel(spp_path)

In [ ]:
spp_data

In [ ]:
stop

In [ ]:
d = dict(zip(spp_data['发货商ID'],spp_data['发货商'],))

d

In [ ]:
conn.keys('ORDER:NEW:TEMP*')

In [ ]:
conn = redis.Redis(**REDIS_MALL_ORDER_DIC)

conn.keys('ORDER:NEW:TEMP*')
src_new = 'ORDER:NEW:TEMP:S0003'
dst_new = src_new.replace(":TEMP","")
conn.sunionstore( dst_new,dst_new,src_new)
conn.delete(src_new)

In [ ]:
# conn.sunionstore('ORDER:NEW',"ORDER:NEW_TEMP",'ORDER:NEW')

# conn.sadd("ORDER:NEW","zero")
# conn.sadd("ORDER:WB","zero")
# conn.sadd("ORDER:NEW_TEMP","zero")